In [67]:
from PIL import Image
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
from utils import *
import torch
from multi_UNet_complex import *
import os
from torchvision import transforms
import cv2

device = torch.device('cpu')


In [68]:
def predict_img(model_path, image_path, output_path):

    ## CREATION OF YOUR MODEL
    net_test = ComplexUNet(4)
    net_test = net_test.to(device)
    net_test.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    # Load the image
    image = Image.open(image_path)

    # Convert the image to grayscale
    if image.mode != 'L':
        image = image.convert('L')

    # Preprocess the image
    x = TF.to_tensor(image)
    x = TF.normalize(x, [0.5], [0.5])
    x = x.unsqueeze(0)  # Add batch dimension

    # Move the tensor to the same device as the model
    x = x.to(device)

    output = net_test(x)

    # The output is the predicted segmentation
    predicted_segmentation = torch.argmax(output.squeeze(), dim=0)

    # Convert the tensor to a numpy array
    predicted_segmentation = predicted_segmentation.detach().cpu().numpy()

    # Map the predicted class to the color code 
    mapping = {0: 0, 1: 85, 2: 170, 3: 255}
    predicted_segmentation = np.vectorize(mapping.get)(predicted_segmentation)

    # Save the predicted segmentation as an image
    predicted_image = Image.fromarray(predicted_segmentation.astype('uint8'))
    predicted_image.save(output_path)

In [69]:
def predict_folder(model_path, input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Liste des fichiers dans le dossier d'entrée
    input_files = os.listdir(input_folder)

    for input_file in input_files:
        input_image_path = os.path.join(input_folder, input_file)

        output_image_path = os.path.join(output_folder, input_file)

        predict_img(model_path, input_image_path, output_image_path)

In [70]:
predict_folder('./models/1_Epoch', './Data all labelled/test/Img', './img/predictions')